In [1]:
import os
import sys

src_path = os.path.abspath('..')
print(src_path)
sys.path.append(src_path)

/home/zw12/MedLink/src


In [2]:
from utils import data_path, set_seed, create_directory, write_json

In [3]:
import json
import os
from itertools import chain
import tqdm
import random
import pickle
import pandas as pd
import numpy as np

In [4]:
set_seed(seed=42)

In [5]:
data_madm = pd.read_csv(os.path.join(data_path, 'mimic3/processed/data_madm.csv'))
data_madm.head()

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
0,10004,M,164713,2182-11-02,2182-11-19,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,E8889 78039 7837 51881 4619 5225 3484 2720 803...,272 780 518 522 348 599 461 401 783 803 507 25...,58 159 233 53 98 131 49 126 95 83 663 136 2603...,52.0
1,10004,M,161106,2183-05-04,2183-05-08,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,E8859 5180 85200 25000 41401 3453 80503 V453,518 852 345 414 E885 250 V45 805,233 49 83 130 2603 231 155 101,52.0
2,1004,F,127794,2108-05-10,2108-05-25,ELECTIVE,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,99702 V5865 29680 44422 37851 4430 9972 5990 5...,296 753 443 997 998 518 588 599 437 E879 378 4...,159 111 131 161 238 99 2616 91 117 657 257 215...,55.0
3,1004,F,177186,2108-06-07,2108-06-09,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,44422 43491 99811 2760 452 2724 4019 2874 2967...,296 272 753 584 998 588 E934 287 452 573 434 4...,157 53 55 238 118 161 98 151 62 215 116 109 26...,56.0
4,10059,M,142582,2150-08-07,2150-08-13,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,CATHOLIC,MARRIED,WHITE,7895 5723 07070 45620 4019 2800 5715 5849 2875,456 280 571 584 287 401 789 572 070,157 6 98 153 151 62 59,69.0


In [6]:
data_madm.SUBJECT_ID = data_madm.SUBJECT_ID.apply(lambda x: str(x))
data_madm.HADM_ID = data_madm.HADM_ID.apply(lambda x: str(x))

In [7]:
print('GENDER', data_madm.groupby('SUBJECT_ID').GENDER.nunique().mean())
print('AGE', data_madm.groupby('SUBJECT_ID').AGE.nunique().mean())
print('INSURANCE', data_madm.groupby('SUBJECT_ID').INSURANCE.nunique().mean())
print('LANGUAGE', data_madm.groupby('SUBJECT_ID').LANGUAGE.nunique().mean())
print('RELIGION', data_madm.groupby('SUBJECT_ID').RELIGION.nunique().mean())
print('MARITAL_STATUS', data_madm.groupby('SUBJECT_ID').MARITAL_STATUS.nunique().mean())
print('ETHNICITY', data_madm.groupby('SUBJECT_ID').ETHNICITY.nunique().mean())

GENDER 1.0
AGE 1.8026696023118205
INSURANCE 1.1242603550295858
LANGUAGE 0.7494151644419981
RELIGION 1.0214669051878353
MARITAL_STATUS 1.0220173386541902
ETHNICITY 1.0399064263107196


In [8]:
from beir.datasets.data_loader import GenericDataLoader


code = 'CCS_CODE'  # code doesn't matter as we only need qurey hadm_id
_data_path = os.path.join(data_path, f'mimic3/processed/mimic3_{code}')
te_corpus, te_queries, te_qrels = GenericDataLoader(_data_path).load(split="test")
va_corpus, va_queries, va_qrels = GenericDataLoader(_data_path).load(split="val")

/home/zw12/miniconda3/envs/pytorch20/lib/python3.8/site-packages/beir/datasets/data_loader.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7267/7267 [00:00<00:00, 447445.79it/s]


In [9]:
print(len(te_qrels))
print(len(va_qrels))

1454
727


In [10]:
te_corpus == va_corpus

True

In [11]:
# merge te and va for downstream processing
for k, v in va_qrels.items():
    assert k not in te_qrels
    te_qrels[k] = v
print(len(te_qrels))

2181


In [12]:
for k, v in va_queries.items():
    assert k not in te_queries
    te_queries[k] = v
print(len(te_queries))

2181


In [13]:
q_hadm_ids = list(te_queries.keys())
len(q_hadm_ids)

2181

In [14]:
c_hadm_ids = list(te_corpus.keys())
len(c_hadm_ids)

7267

In [15]:
matches = ['GENDER', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY']

In [16]:
q_to_can = {}
for q_hadm_id in tqdm.tqdm(q_hadm_ids):
    row = data_madm[data_madm.HADM_ID == q_hadm_id]
    matched_pd = pd.merge(data_madm, row, on=matches, how='inner')
    matched_pd = matched_pd[matched_pd.AGE_x <= row.AGE.iloc[0]]
    can_hadm_ids = list(set(matched_pd.HADM_ID_x.tolist()).intersection(set(c_hadm_ids)))
    q_to_can[q_hadm_id] = can_hadm_ids

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2181/2181 [00:16<00:00, 132.34it/s]


In [17]:
len(q_to_can)

2181

In [18]:
tmp = [len(v) for v in q_to_can.values()]
print('min', min(tmp))
print('max', max(tmp))
print('mean', np.mean(tmp))
print('std', np.std(tmp))

min 0
max 267
mean 32.54057771664374
std 48.01771442033842


any non-match?

In [19]:
c = 0
for k, v in q_to_can.items():
    if len(v) == 0:
        print(k)
        c += 1
c

102584
103106
104699
109386
109940
110408
111911
115106
115735
115850
117548
119286
119367
120594
122084
123538
124442
125924
129791
131022
132160
135006
137439
140109
148582
149406
149473
154121
154417
157053
158598
160104
161002
161892
162322
163837
164343
166755
167377
168178
168295
168961
170525
171639
176105
178034
180859
181175
181404
181880
185457
185567
187061
191197
197509
199478
100069
101728
103169
107753
111425
115932
118047
126802
127135
131258
135593
139654
145567
146572
147174
148918
156448
156641
159513
163788
167702
168792
170957
171144
174404
183768
187001
188769
195430
196404
198056
198353
199339


89

In [20]:
data_madm[data_madm.HADM_ID == '102584']

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
16453,79015,M,102584,2194-12-01,2194-12-05,EMERGENCY,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Medicare,ENGL,EPISCOPALIAN,SINGLE,WHITE,59960 9693 5119 5989 5070 7587 9663 2720 2724 ...,305 599 E950 780 710 788 251 518 598 969 783 2...,259 58 662 163 53 55 242 241 131 660 217 162 1...,37.0


In [21]:
data_madm[data_madm.SUBJECT_ID == '79015']

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
16452,79015,M,155865,2194-01-25,2194-02-19,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,EPISCOPALIAN,SINGLE,WHITE,8074 80229 27800 4619 8600 99731 E8147 9348 85...,802 825 807 278 461 728 860 958 518 453 873 04...,58 233 2607 238 131 234 118 126 228 211 2 230 ...,36.0
16453,79015,M,102584,2194-12-01,2194-12-05,EMERGENCY,CLINIC REFERRAL/PREMATURE,SHORT TERM HOSPITAL,Medicare,ENGL,EPISCOPALIAN,SINGLE,WHITE,59960 9693 5119 5989 5070 7587 9663 2720 2724 ...,305 599 E950 780 710 788 251 518 598 969 783 2...,259 58 662 163 53 55 242 241 131 660 217 162 1...,37.0


In [22]:
data_madm[data_madm.HADM_ID == '178034']

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
1200,12446,F,178034,2186-11-03,2186-11-14,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,1985 V4282 2859 V103 5780 591 2875 E9331,V10 578 E933 285 287 198 V42 591,259 42 24 161 153 62 2617 59,55.0


In [23]:
data_madm[data_madm.SUBJECT_ID == '12446']

,SUBJECT_ID,GENDER,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,ICD9_FULL_CODE,ICD9_3DIGIT_CODE,CCS_CODE,AGE
1199,12446,F,150404,2185-07-12,2185-07-27,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DISC-TRAN CANCER/CHLDRN H,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,1986 1985 19882 5934 99859 V103 5990 9975 591,V10 997 998 593 198 599 591,42 159 238 24 161,54.0
1200,12446,F,178034,2186-11-03,2186-11-14,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,1985 V4282 2859 V103 5780 591 2875 E9331,V10 578 E933 285 287 198 V42 591,259 42 24 161 153 62 2617 59,55.0


solve non-match

In [24]:
tmp = [len(v) for v in q_to_can.values()]
mean_sample_size = int(np.mean(tmp))
mean_sample_size

32

In [25]:
for k, v in q_to_can.items():
    if len(v) == 0:
        q_to_can[k] = random.sample(c_hadm_ids, mean_sample_size)

In [26]:
c = 0
for k, v in q_to_can.items():
    if len(v) == 0:
        print(k)
        c += 1
c

0

are true value sampled?

In [27]:
c = 0
for q_id, c_ids in te_qrels.items():
    for c_id in c_ids.keys():
        if c_id not in q_to_can[q_id]:
            print(q_id)
            c += 1
c

100915
101028
101092
101322
101993
102218
102500
102584
102679
102996
103106
104647
104699
105135
105764
106291
106963
107029
107032
109309
109386
109486
109935
109940
109997
110408
110868
111911
111936
112278
112623
113156
115106
115185
115735
115845
115850
116163
117548
118208
118250
119185
119286
119367
120594
121100
121729
121979
122084
122970
123322
123538
124006
124124
124134
124212
124442
124735
125924
126811
127484
127903
129236
129668
129791
130619
131022
131449
131592
131981
132967
133125
133142
133304
134510
134638
134918
135006
136614
137241
137321
137439
137889
138497
138983
139397
139476
139914
140032
140109
140246
140890
141081
142471
142696
142728
143237
145555
145683
146178
147841
148278
148582
148606
149243
149406
149473
149781
149970
150155
150367
151009
151102
152321
152516
153101
154010
154086
154121
154417
155256
155277
155442
156938
157053
158067
158513
158598
160104
160904
160946
160971
161002
161091
161892
162322
162937
163733
163837
164119
164343
164706
164967

353

remove true value

In [28]:
for k, v in q_to_can.items():
    c_ids = te_qrels[k]
    for c_id in c_ids.keys():
        if c_id in v:
            v.remove(c_id)
    q_to_can[k] = v

In [29]:
c = 0
for q_id, c_ids in te_qrels.items():
    for c_id in c_ids.keys():
        if c_id in q_to_can[q_id]:
            print(q_id)
            c += 1
c

0

In [30]:
tmp = [len(v) for v in q_to_can.values()]
print('min', min(tmp))
print('max', max(tmp))
print('mean', np.mean(tmp))
print('std', np.std(tmp))

min 0
max 266
mean 33.00825309491059
std 47.551491680570436


In [31]:
write_json(os.path.join(data_path, 'mimic3/processed/candidate.json'), q_to_can)